# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** K

**Names:**

* Kim Lan Phan Hoang
* Robin Lang

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl

from nltk.stem.wordnet import WordNetLemmatizer

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')
punctuation = '.?!,;:-–()[]{}"/\''

## Exercise 4.1: Pre-processing

### Pre-process the corpus to create bag-of-words representations of each document

In [2]:
MIN = 10
MAX = 2000

# note: downloading wordnet is required
# command: python -m nltk.downloader
#   in the console
lemmatizer = WordNetLemmatizer()

In [3]:
# http://locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

In [4]:
courses_proc = {}
words = {}
cids = []

for c in courses:
    cid = c['courseId']
    cids.append(cid)
    
    # transfer all to lowercase, remove punctuation
    desc = c['description'].lower().translate(str.maketrans('', '', punctuation))
    # remove stopwords
    desc_proc = [lemmatizer.lemmatize(word, pos='v') for word in desc.split() if word not in stopwords]
    desc_2grams = find_ngrams(desc_proc, 2)
    
    desc_proc.extend(desc_2grams)
    
    # create a dict of all words
    for w in desc_proc:
        if w in words:
            words[w] += 1
        else:
            words[w] = 1
    
    # not needed ?
    #name = c['name']
    #courses_proc[cid] = desc_proc
    
    if cid in courses_proc:
        # some courseIds appear multiple times
        # for those, we decided to append the desctiptions to each other
        courses_proc[cid].extend(desc_proc)
    else:
        courses_proc[cid] = desc_proc

In [ ]:
words_filtered = []

# most and least frequent words to filter
for w in words:
    if words[w] > MIN or words[w] < MAX:
        words_filtered.append(w)

In [ ]:
courses_proc2 = {}

for c in courses_proc:
    courses_proc2[c] = [word for word in courses_proc[c] if word in words_filtered]

### Explain which kinds of cleaning you implemented and why

* make sure every courseId only appears once. If it appears more than once, append the two descriptions
* Put all cheacters to lowercase, to avoid the same word being considered different. Additionally, all words in stopwords.pkl are in lowercase, so this ensures all words are removed correctly
* Remove punctuation, for the same reason as above. Only the words are important, not which one is before a comma.
* Remove stopwords, as these don't carry any information about the content of the document
* filter most and least frequent words, as they only mess up the results when trying to find similarities between documents. commomn words will be in almost every document, rare ones in almost none.
* Lemmatization, to detect only words with different meaning. Identical words, such as "is" and "are" will automatically be transformed to "be", the same is true with multiples ("documents" -> "document").

### Print the terms in the pre-processed description of the $9^{th}$ class in alphabetical order.

In [ ]:
print(sorted(courses_proc2[courses[9]['courseId']]))

## Exercise 4.2: Term-document matrix

### Construct an M×N term-document matrix X, where M is the number of terms and N is the number of documents. The matrix X should be sparse. You are not allowed to use libraries for this task.

In [ ]:
# mapping from word to its index
words_index = {}
i = 0

for w in words_filtered:
    words_index[w] = i
    i += 1

In [ ]:
# mapping from courseId to its index
courses_index = {}
i = 0

for w in courses_proc2:
    courses_index[w] = i
    i += 1

In [ ]:
M = len(words_filtered)
N = len(courses_proc2)

In [ ]:
X = np.ndarray((M, N))

for c in courses_proc2:
    for t in courses_proc2[c]:
        X[words_index[t]][courses_index[c]] += 1
        if courses_index[c]==0:
            i += 1

### Print the 15 terms in the description of the $9^{th}$ class with the highest TF-IDF scores.

In [ ]:
TF = np.array([x / x.max() for x in X.T])

In [ ]:
IDF = np.ndarray(M)
for i in range(len(X)):
    count = 0
    for j in X[i]:
        if j > 0:
            count += 1
    IDF[i] = np.log2(N / count)

In [ ]:
TFIDF = np.array([x * IDF for x in TF])

In [ ]:
IFIDF_9 = TFIDF[courses_index[courses[9]['courseId']]]

In [ ]:
TFIDF_9_top15 = np.argsort(-IFIDF_9)[:15]

In [ ]:
print("TOP 15 TERMS FROM TFIDF")
for i in TFIDF_9_top15:
    for k, v in words_index.items():
        if v == i:
            print(" ", IFIDF_9[i], ":", k)

### Explain where the difference between the large scores and the small ones comes from.

TF gives the importance if a word in a document, between 0 and 1. 1 is the most frequent word, 0 means they never appear.

IDF gives the frequency of a word in the corpus, bigger than 0. 0 means the word is in every description, infinity means it's in none of them.

TFIDF then is simply the mutliplication of the two, where a high TFIDF means that word is frequent within the desctiption, but rare within the corpus.

## Exercise 4.3: Document similarity search

### Search for "markov chains" and "facebook".

In [ ]:
markov_index = words_index['markov']
chains_index = words_index['chains']
facebook_index = words_index['facebook']

### Display the top five courses together with their similarity score for each query.

### What do you think of the results? Give your intuition on what is happening.